---
title: "Using the Heat Equation to model a real-world system (Part 4)"
description: "Where or where did the physics go?"
author: "Steven Wolf"
date: "08/01/2025"
number-sections: false
categories:
  - Heat Equation
  - Modeling
  - Classroom Models vs Realistic Models
  - Yapping
execute: 
  messages: false
  warning: false
jupyter: python3
draft:
  true
---

## Where's the physics?

[Previously](HeatEqnReset2.qmd), I solved the 1D bar problem with Robin boundary conditions. And I promised to make this algorithmic using the [FENICS Project code](https://fenicsproject.org/), and [I have done that](HeatEqnReset3). Actually, [I've done that twice](HeatEqnRobinRod.qmd), no, make that [three times](HeatEqnRobinSlab.qmd), but we need to take a bit of an aside. Specifically, it's time to do some physics. Let's look at the heat equation that I presented previously through the eye of a physicist.

$$
\frac{\partial u(\vec{x},t)}{\partial t} - \nabla^2 u(\vec{x},t) = f(\vec{x},t)
$$

A physicist looks at this and says this equation doesn't work because of *units*. Let's look at each term of the above equation:

1. $\frac{\partial u(\vec{x},t)}{\partial t}$: If $u(\vec{x},t)$ is the temperature as a function of space and time, it has units of temperature. Let's assume that is Kelvin. Taking the partial derivative with respect to time, means the units of this term is Temperature/Time or K/s.
2. $\nabla^2 u(\vec{x},t)$: Since $\nabla^2$ is a second derivative with respect to a length $x$, then the units of this term is Temperature/(Length)$^2$ or K/(m$^2$).
3. $f(\vec{x},t)$ is called an "external power generation term". So that would imply units of Power or Watts (W).

None of these terms have the same units. So a physicist says that this equation is effectively asking you to add \$5 + 3 apples + 2 m, and looks at you like you are crazy. So let's resolve that issue.

### A mathematical straw-man
Note: I'm making a *bit* of a straw-man argument here. Note that when mathematicians are solving differential equations, they are often most interested in the behavior of the functions. Units are a bit of a sticky wicket with mathematicians, and the way that physicists do dimensional analysis isn't mathematically accurate. A mathematician would say that $\vec{x}\in\mathbb{R}^3$ and $t\in\mathbb{R}^+$ (among other things), which are formal definitions to guide the mathematician. It is when we apply these mathematical constructs to describe physical system that we map these constructs onto physical quantities like time and space, which we (sometimes, inconveniently) measure with different units. 

The issue with units is that they are heavily context dependent. I'll speak about SI units here. There are some systems for which the meter is a perfectly good unit of measure. For example, Track and Field records their field event records in meters, and the numbers are "nice". For example the long jump world record is 8.95 meters. This is much more convenient to write down than 8950000000000000 Fermi, or 0.000000000059827 AU. But the Fermi is quite convenient for the nuclear physicist to use to measure the size of the typical nucleus (1 fm = $10^{-15}$ m). And the AU is used by planetary scientists to measure the distance between objects in the solar system, since 1 AU = $1.496\times 10^{11}$ m is the average distance between the earth and the sun. And a Cosmologist would prefer to measure distances in light-years.  Indeed this convenience standard is a hidden "perk" of using units like inches or feet--a perk with a drawback. The perk being everyone has a foot, or an inch (distance between 1st and 2nd knuckle on pointer finger), or a cubit (distance between elbow and fingertip). The drawback is obvious--my foot might change size over a long time period and ***YOUR*** foot and ***MY*** foot aren't the same size.

### Can you fix it?
Ok, so I'm yapping again. Let's fix the problem. And let's take a lesson from middle school math classes. A 6th grader might be asked to add 2 m + 35 cm. The successful 6th grade will correctly convert one of the measurements to the other unit system and add the result as usual getting 2.35 m or 235 cm. They multiply by the appropriate conversion factor:
$$
\frac{100 \text{cm}}{1 \text{m}} \quad \text{or} \quad \frac{1 \text{m}}{100 \text{cm}}
$$
So this gives us a formula for fixing the problem. When trying to add things with different units, multiply one by the appropriate conversion factor to make the units the same.

### Heat Equation with units
So if we do this to the heat equation, we get the following
$$
\frac{\partial u(\vec{x},t)}{\partial t} - \alpha \nabla^2 u(\vec{x},t) = \frac{\dot{Q}(\vec{x},t)}{c_p \rho}
$$
Let's detail what all of these quantities are, and the units we use:
1. $\vec{x}$ is a vector length measured in meters (m).
2. $t$ is a time measured in seconds (s).
3. $u(\vec{x},t)$ is a function for which the inputs are the location in space and the time, and the output is a temperature measured in Kelvin (K).
4. $\alpha$ is the thermal diffusivity of the material and is measured in m$^2$/s. Note, that if the system is made up of many materials, or materials that change with time, this can also be a function of space and time.
5. $\dot{Q}(\vec{x},t)$ is a function for which the inputs are the location in space and the time, and the output is a power density measured in W/m$^3$. Note that 1 W = 1 J/s.
6. $c_p$ is the specific heat capacity of a material and is measured in J/(kg K).
7. $\rho$ is the (mass) density and is measured in kg/m$^3$.

If we do this, we find that each term in the heat equation has units of K/s.

We also need to figure out the units for the boundary conditions.
1. Dirichlet Condition - The boundary temperature is given ***in Kelvin*** at each point in space/time. So for $\vec{x}\in\Gamma_D$:
$$
u(\vec{x},t) = u_D(\vec{x},t)
$$
2. Neumann Condition - The Heat Flux $(\Phi)$ ***in W/m$^2$*** is given at the boundary at each point in space/time. So for $\vec{x}\in\Gamma_N$:
$$
-\kappa \frac{\partial u(\vec{x},t)}{\partial n} = \Phi(\vec{x},t)
$$
where $\kappa$ is the thermal conductivity measured in W/(m K).
3. Robin Condition - The Heat flux at the boundary is related to the temperature at the boundary. So for $\vec{x}\in\Gamma_R$:
$$
-\kappa \frac{\partial u(\vec{x},t)}{\partial n} = h(u(\vec{x},t) - s_{\text{ext}})
$$
where $u(\vec{x},t)$ and $\kappa$ are defined as before, $h$ is the heat transfer coefficient measured in W/(m$^2$ K), and $s_{\text{ext}}$ is the temperature of the exterior measured in K.

## Finite Element formulation of the Heat Equation with units:
So we will repeat the previous analysis of the heat equation, this time including units correctly. Assume the initial condition is:
$$
u(\vec{x},0) = u_I(\vec{x})
$$

### Discretize time
When we discretize the time variable as before:
$$
u^k(\vec{x}) = u(\vec{x},t_k) \quad \dot{Q}^k(\vec{x}) = \dot{Q}(\vec{x}, t_k)
$$
we rewrite the initial condition:
$$
u^0(\vec{x}) = u_I(\vec{x})
$$
and the heat equation becomes:
$$
\frac{u^k(x)-u^{k-1}(x)}{\Delta t} - \alpha \frac{d^2 u^k(x)}{dx^2} = \frac{\dot{Q}^k(\vec{x})}{c_p \rho}
$$

### Weak formulation:
Now convert to the weak formulation for the initial condition:
$$
\int_\Omega u^0(\vec{x}) v(\vec{x}) d^3x = \int_\Omega u_I(\vec{x}) v(\vec{x}) d^3x
$$
This allows us to define two functions:
$$
a_0(u,v) = \int_\Omega u^0(\vec{x}) v(\vec{x}) d^3x 
$$
and
$$
L_0(v) = \int_\Omega u_I(\vec{x}) v(\vec{x}) d^3x
$$

Similarly, we can convert to the weak formulation for the heat equation:

$$
\begin{align*}
\int_\Omega & \left(u^k(\vec{x})v(\vec{x}) - \alpha \Delta t \frac{d^2 u^k(x)}{dx^2} v(\vec{x})\right) d^3x \\
= &\int_\Omega \left(u^{k-1}(\vec{x})v(\vec{x}) + \Delta t \frac{\dot{Q}^k(\vec{x})}{c_p \rho} v(\vec{x})\right)d^3x
\end{align*}
$$

Integrating the 2nd term on the LHS by parts, and using the definitions of each kind of boundary, we get:

$$
\begin{align*}
\int_\Omega &\left(u^k(\vec{x})v(\vec{x}) + \alpha \Delta t \nabla u^k(\vec{x}) \cdot \nabla v(\vec{x})\right) d^3x  + \sum_l \int_{\Gamma_R^l} \frac{h_l^k}{c_p \rho} u^k(\vec{x}) v(\vec{x}) ds \\
= & \int_\Omega \left(u^{k-1}(\vec{x})v(\vec{x}) + \Delta t \frac{\dot{Q}^k(\vec{x})}{c_p \rho} v(\vec{x})\right)d^3x + \sum_l \int_{\Gamma_R^l} \frac{h_l^k}{c_p \rho} s^k_l(\vec{x}) v(\vec{x}) ds \\
& + \sum_j \int_{\Gamma_N^j} \frac{g_j^k}{c_p \rho} v(\vec{x}) ds
\end{align*}
$$

So we can define the bilinear form:
$$
a(u^k, v) = \int_\Omega \left(u^k(\vec{x})v(\vec{x}) + \alpha \Delta t \nabla u^k(\vec{x}) \cdot \nabla v(\vec{x})\right) d^3x  + \sum_l \int_{\Gamma_R^l} \frac{h_l^k}{c_p \rho} u^k(\vec{x}) v(\vec{x}) ds
$$
and the linear form
$$
\begin{align*}
L(v) = &\int_\Omega \left(u^{k-1}(\vec{x})v(\vec{x}) + \Delta t \frac{\dot{Q}^k(\vec{x})}{c_p \rho} v(\vec{x})\right)d^3x \\
 &+ \sum_l \int_{\Gamma_R^l} \frac{h_l^k}{c_p \rho} s^k_l(\vec{x}) v(\vec{x}) ds + \sum_j \int_{\Gamma_N^j} \frac{g_j^k}{c_p \rho} v(\vec{x}) ds
\end{align*}
$$
